In [ ]:
import os
import tensorflow as tf
import numpy as np
import utilities as u

from keras.models import load_model
from pathlib import Path

# Ignore Tensorflow Warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'

# tf.config.set_visible_devices([], 'GPU')
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
# Model information (used later for image output)
model_dir = "Surrogate"

# Additional run info
variables = ["Hs", "Tm02", "Dir"]
grid = (160, 160)

# Define the point where the time series is extracted (in pixel)
xcoord = [69, 86, 69]
ycoord = [94, 104, 77] 

# Test data serials
sample_start = 8761
sample_end = 17496
serial_test = np.arange(sample_start, sample_end+1, 1)

# Training data serials
sample_ref_start = 24
sample_ref_end = 8760
serial_ref = np.arange(sample_ref_start, sample_ref_end+1, 1)

dim = 1

nfreq = 32
ntheta = 24

# Create output directory if needed
path_dir = Path("Data/Time_Series")

if not path_dir.exists():
    path_dir.mkdir()

In [ ]:
# Load normalization constants
X_max, X_min = np.load("Data/Xmax_Xmin_2018_spectrum.npy")
bat_max, bat_min = np.load("Data/bat_max_bat_min_2018_spectrum.npy")

In [ ]:
for k, var in enumerate(variables): # Load best model of the run

    # Prepare file locations for reference and input data
    fname_HR = f'Data/HR/{var}/BaskCoast_{var.upper()}_{{}}.npy'
    fname_spec = 'Data/Spectrum/BaskCoast_{}.npy'
    fname_bat = 'Data/Bathymetry/bat.npy'

    fdir = Path(f"Models/{model_dir}")
    fmodel = fdir / f"Model_Inp_{var}.hdf5"
    model = load_model(fmodel)
    
    # Load training data and shift directional data if needed
    X, y = u.load_data((fname_HR, fname_spec), serial_test, var, convert=False,
                       grid=(nfreq, ntheta))

    # Load bathymetry and tile to right length
    bat_tot = np.load(fname_bat).reshape((1, *grid, dim))
    bat_tot = np.tile(bat_tot, (len(serial_test), 1, 1, 1))
    
    # Normalize data
    X = (X - X_min) / (X_max - X_min)
    bat_tot = (bat_tot - bat_min) / (bat_max - bat_min)
    
    prediction = model.predict([X, bat_tot])
    
    if var == "Dir":     
        # Shift directional data back
        prediction = (prediction + 255)%360
        
    for point in zip(xcoord, ycoord):
        xi = point[0]
        yi = point[1]
        
        # Load saved array, if calculation was already done before
        path_dir = Path("Data/Time_Series")
        path_arr = path_dir / f"Var_{var}_posxy_{xi}_{yi}.npy"
        
        # For low-resolution input use array full of nans
        lr_input = np.zeros_like(prediction)
        lr_input[:] = np.nan

        # Save the computed data
        if not path_dir.exists():
            path_dir.mkdir(parents=True)

        np.save(path_arr, np.row_stack(
            (prediction[:, xi, yi, 0],
             y[:, xi, yi, 0],
             lr_input[:, xi, yi, 0])))

    del X, y, bat_tot, prediction
    
    # Extract time series for HR training data in similar fashion
    # Prepare file locations for reference
    fname_HR = f'Data/HR/{var}/BaskCoast_{var.upper()}_{{}}.npy'

    # Define output directory
    path_dir = Path("Data/Time_Series")
    path_arr = path_dir / f"Ref_Var_{var}_posxy_{xi}_{yi}.npy"

    y = np.zeros((len(serial_ref), grid[0], grid[1], dim))

    # Load reference training data
    for i, s in enumerate(serial_ref):
        y[i, :, :, 0] = np.load(fname_HR.format(s))

    # Extract time series at given location
    ref = y[:, xi, yi, 0]

    np.save(path_arr, ref)
